In [60]:
using RDatasets, Test, GLM, LinearAlgebra, GeneralizedCopula, StatsBase
using LinearAlgebra: BlasReal, copytri!

dyestuff = dataset("lme4", "Dyestuff")
groups = unique(dyestuff[!, :Batch])
n, p, m = length(groups), 1, 1

d = Bernoulli()
D = typeof(d)

eta = zscore(dyestuff[!, :Yield])
mu = GLM.linkinv.(canonicallink(d), eta)

y = zeros(length(mu))
function __get_distribution(dist::Type{D}, μ) where D 
    return dist(μ)
end

for i in eachindex(y)
  y[i] = rand(__get_distribution(D, mu[i]))
end

dyestuff[!, :y_logistic] = y
dyestuff[!, :normal] = rand(30)
dyestuff

,Batch,Yield,y_logistic,normal
,Cat…,Int32,Float64,Float64
1,A,1545,0.0,0.26731
2,A,1440,0.0,0.396773
3,A,1440,0.0,0.333118
4,A,1520,0.0,0.831202
5,A,1580,1.0,0.517746
6,B,1540,0.0,0.0339431
7,B,1555,0.0,0.890732
8,B,1490,1.0,0.227477
9,B,1560,1.0,0.446638


In [62]:
glmm1 = MixedModels.fit!(GeneralizedLinearMixedModel(@formula(y_logistic ~ 1 + normal + (1|Batch)), dyestuff, Bernoulli()), fast = true)

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  y_logistic ~ 1 + normal + (1 | Batch)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 41.4512

Variance components:
         Column   VarianceStd.Dev.
Batch (Intercept)  0 0

 Number of obs: 30; levels of grouping factors: 6

Fixed-effects parameters:
─────────────────────────────────────────────────────
              Estimate  Std.Error    z value  P(>|z|)
─────────────────────────────────────────────────────
(Intercept)  -0.254183   0.778091  -0.326675   0.7439
normal        0.522642   1.41176    0.370207   0.7112
─────────────────────────────────────────────────────

In [63]:
loglikelihood(glmm1)

-20.725592578236807

In [67]:
groups = unique(dyestuff[!, :Batch])
n, p, m = length(groups), 1, 1
d = Bernoulli()
D = typeof(d)
gcs = Vector{GLMCopulaVCObs{Float64, D}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = dyestuff[!, :Batch] .== grp
    ni = count(gidx)
    y = Float64.(dyestuff[gidx, :y_logistic])
    normal = Float64.(dyestuff[gidx, :normal])
    X = [ones(ni, 1) normal]
    V = [ones(ni, ni)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β

1 0.0 -20.936129595976524 23
2 -20.936129595976524 -20.725596404036935 5
3 -20.725596404036935 -20.72559257823683 5
4 -20.72559257823683 -20.725592578236807 5
gcm.β = [-0.25418263434611227, 0.5226417589863994]


2-element Array{Float64,1}:
 -0.25418263434611227
  0.5226417589863994

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [68]:
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
@show gcm.Σ
GeneralizedCopula.loglikelihood3!(gcm, true, true)

gcm.Σ = [3.556059762903249e-5]


-20.725606809084436

In [69]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 300, derivative_test = "first-order", hessian_approximation = "limited-memory"))

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          1] = -8.9966177535032177e+00    ~ -8.5811015008394861e+02  [ 9.895e-01]
* grad_f[          2] = -4.3258149595287660e+00    ~ -9.0043125865690854e+03  [ 9.995e-01]

Derivative checker detected 2 error(s).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
    

GLMCopulaVCModel{Float64,Bernoulli{Float64}}(GLMCopulaVCObs{Float64,Bernoulli{Float64}}[GLMCopulaVCObs{Float64,Bernoulli{Float64}}([0.0, 0.0, 0.0, 0.0, 1.0], [1.0 0.2673099962929948; 1.0 0.3967732912373647; … ; 1.0 0.831202167151599; 1.0 0.5177461341313283], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [-1.4887357466053441, -0.6758522612839774], [-0.4721848678021601 -0.12621973526180366; -0.48843283056884634 -0.1937971017331833; … ; -0.5471522116595678 -0.45479410409322324; -0.49591353192880094 -0.25675731401954976], [NaN], [2.1819477796e-314], [-1.2466092630626646 -0.5844440375958104; -0.5844440709247736 -0.32317829751341615], [2.216718183e-314], [-0.9443697356043202, -0.9768656611376928, -0.9607505388544347, -1.094304423319136, 0.991827063857602], [2.5], [4.453510579774172], [5.0 2.3461498442505953; 2.3461498442505953 1.2988095529485373], [-2.9844632950579815, -2.9844632950579815, -2.9844632950579815, -2.9844632950579815, -2.9844632950579815], [

In [70]:
@show loglikelihood3!(gcm, true, true)
@show gcm.β
@show gcm.∇β

loglikelihood3!(gcm, true, true) = -20.72559297010178
gcm.β = [-0.2541820795091232, 0.5226405317923359]
gcm.∇β = [-3.1514930665110796e-9, -5.887083431854734e-9]


2-element Array{Float64,1}:
 -3.1514930665110796e-9
 -5.887083431854734e-9

In [71]:
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
GeneralizedCopula.loglikelihood3!(gcm, true, true)
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))

1 0.0 -20.936129595976524 23
2 -20.936129595976524 -20.725596404036935 5
3 -20.725596404036935 -20.72559257823683 5
4 -20.72559257823683 -20.725592578236807 5
gcm.β = [-0.25418263434611227, 0.5226417589863994]
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   ine

GLMCopulaVCModel{Float64,Bernoulli{Float64}}(GLMCopulaVCObs{Float64,Bernoulli{Float64}}[GLMCopulaVCObs{Float64,Bernoulli{Float64}}([0.0, 0.0, 0.0, 0.0, 1.0], [1.0 0.2673099962929948; 1.0 0.3967732912373647; … ; 1.0 0.831202167151599; 1.0 0.5177461341313283], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [-1.4887082066864148, -0.6758387973397937], [-0.4721850933296278 -0.12621979554755022; -0.48843290030814546 -0.19379712940387453; … ; -0.5471516749998886 -0.45479365802053473; -0.49591361628472475 -0.2567573576945032], [NaN], [2.1819477796e-314], [-1.2466092832622706 -0.5844440953519666; -0.5844440959953953 -0.32317832518282735], [2.216718183e-314], [-0.9443701866592555, -0.9768658006162909, -0.9607508342074047, -1.0943033499997772, 0.9918272325694494], [2.5], [4.453509516873443], [5.0 2.3461498442505953; 2.3461498442505953 1.2988095529485373], [-2.9844629389132793, -2.9844629389132793, -2.9844629389132793, -2.9844629389132793, -2.9844629389132793],

In [72]:
@show loglikelihood3!(gcm, true, true)
@show gcm.β
@show gcm.∇β

loglikelihood3!(gcm, true, true) = -20.725594449692284
gcm.β = [-0.25417974151900485, 0.5226353589980361]
gcm.∇β = [1.279188670544329e-7, 3.1227224450702806e-7]


2-element Array{Float64,1}:
 1.279188670544329e-7
 3.1227224450702806e-7